# Video next to Skeleton64 next to Skeleton20  

In [1]:
import csv
def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break

In [2]:
import pandas as pd
import os
import numpy as np

ANNOTATIONS_PATH = 'data'
REPROCESSED_PATH = ANNOTATIONS_PATH+'/reprocessed'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#annotations_file = ANNOTATIONS_PATH+'/annotationsVSingle.txt'
annotations_file = ANNOTATIONS_PATH+'/annotationsVExamples.txt' #'/annotationsVSingle.txt'

with open(annotations_file,'r') as file:
    annotations = file.read().splitlines()[1:]

sampleAnno = 1

if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
    folder, trial, fragId, _, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
else:
    folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]

reprocessed_file = os.path.join(REPROCESSED_PATH, folder, trial+'_frag'+fragId+'.csv')
reducedMarkersTable = pd.read_csv(reprocessed_file)
raw_file = os.path.join(RAW_PATH, folder, trial+'.tsv')
with open(raw_file,'r') as file:
    numMarkers, columnNames, readerBuffer = custom_header_reader(file)
    fullMarkersTable = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
fullMarkersTable.iloc[:,3:] = (fullMarkersTable.iloc[:,3:] - fullMarkersTable.iloc[:,3:].min(axis=None))/(fullMarkersTable.iloc[:,3:].max(axis=None)-fullMarkersTable.iloc[:,3:].min(axis=None))
fullMarkersTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6196 entries, 0 to 6195
Columns: 195 entries, Frame to LPLM Z
dtypes: float64(193), int64(1), object(1)
memory usage: 9.2+ MB


TODO improve

In [14]:
import cv2
import numpy as np
from tqdm.notebook import tqdm
import os

# Load a pre-trained face detection model (e.g., Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
half_body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')


video_path = '/'.join(os.getcwd().split('/')[:-2]+['docs','presentation','resources','_'.join([folder,trial])+'.mp4'])

vidcap = cv2.VideoCapture(video_path[:-4]+'_'+str(100)+'fps.mp4')

# Get video details
fps = int(vidcap.get(cv2.CAP_PROP_FPS))
width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define a function to censor faces
def censor_faces(frame):
    gray = cv2.cvtColor(frame[height//4:height//4+int(height/2.5),int(width/7.5):int(width/7.5)+int(width/2.1)], cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.1, minNeighbors=5, minSize=(15, 15), maxSize=(40,40))
    if not any(static_face_censor_delay) and len(faces) == 0:
        gray = cv2.addWeighted(gray, 3.1, np.zeros_like(gray), 0, 0)
        half_body = half_body_cascade.detectMultiScale(gray, scaleFactor=1.01, minNeighbors=5, minSize=(120, 120), maxSize=(200,200))
        return *apply_censoring(frame,half_body,is_body=True), half_body
    return *apply_censoring(frame,faces), faces

def apply_censoring(frame, bb, is_body=False):
    for (x, y, w, h) in bb:
        # Apply censorship here, e.g., pixelation
        if is_body:
            x += int(width/7.5) + 30 ; y += height//4 + 40
            w = 60; h = 50
        else:
            enlarger = 4
            x += int(width/7.5) - enlarger; y += height//4 - enlarger
            w += 4*enlarger; h += 4*enlarger
        roi = frame[y:y+h,x:x+w]
        roi = cv2.GaussianBlur(roi, (23, 23), 9)
        frame[y:y+h, x:x+w] = roi
    return frame, is_body

static_face_window = 400
static_face_censor_delay = np.zeros(static_face_window, bool)
moving_face_window = 300
moving_face_censor_delay = np.zeros(moving_face_window, bool)

last_faces = None

if not os.path.isfile(video_path[:-4]+'_face_censored.mp4') or True:
    # Create an output video writer
    out = cv2.VideoWriter(video_path[:-4]+'_face_censored.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    _, frame = vidcap.read()
    faces = [[477, 75, 30, 30]]
    frame,_ = apply_censoring(frame, faces)
    censored = True
    is_body = False
    for i in tqdm(range(1,num_frames), 'Blurring face',miniters=20):
        if is_body:
            moving_face_censor_delay[i%moving_face_window] = censored
            static_face_censor_delay[i%static_face_window] = False
        else:
            static_face_censor_delay[i%static_face_window] = censored
            moving_face_censor_delay[i%moving_face_window] = False

        if censored:
            last_faces = faces
        if not censored and ((not is_body and any(static_face_censor_delay)) or (is_body and any(moving_face_censor_delay))):
            frame,_ = apply_censoring(frame, last_faces, is_body)
        #frame = cv2.rectangle(frame, (int(width/7.5), height//4), (int(width/7.5) + int(width/2.1), height//4 + int(height/2.5)), (255, 0, 0), 2)
        # Add the text to the image
        frame = cv2.putText(frame, str(i), (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        # Write the frame to the output video
        out.write(frame)

        _, frame = vidcap.read()
        # Apply censorship to the frame
        frame, is_body, faces = censor_faces(frame)
        censored = len(faces) > 0
        #if i > 3000: break

    out.release()
vidcap.release()


Blurring face:   0%|          | 0/6195 [00:00<?, ?it/s]

In [27]:
cms = [[[48,3],[3,6]],
       [[51,2],[6,1]],
       [[44,9],[7,0]],
       [[51,3],[4,2]],
       [[52,3],[4,1]],
       [[50,5],[5,0]],
       [[53,2],[2,3]],
       [[54,2],[4,0]],
       [[57,0],[3,0]],
       [[56,2],[2,0]],
       [[58,0],[2,0]],
       [[58,0],[2,0]],
       [[59,0],[1,0]],
       [[59,0],[1,0]],
       [[59,0],[1,0]]]
for cm in cms:
    if cm[1][1] == cm[0][1] == 0:
        print(cm[0][0]/(cm[0][0]+cm[1][0])/2)
    else:
        print((cm[1][1]/(cm[0][1]+cm[1][1])+(cm[0][0]/(cm[0][0]+cm[1][0])))/2)

0.803921568627451
0.6140350877192983
0.43137254901960786
0.6636363636363636
0.5892857142857143
0.45454545454545453
0.7818181818181817
0.46551724137931033
0.475
0.4827586206896552
0.48333333333333334
0.48333333333333334
0.49166666666666664
0.49166666666666664
0.49166666666666664


In [5]:
import cv2
import os
from tqdm.notebook import tqdm

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)) 
  
# initializing subtractor  
fgbg = cv2.createBackgroundSubtractorMOG2() 

video_path = '/'.join(os.getcwd().split('/')[:-2]+['docs','presentation','resources','_'.join([folder,trial])+'.mp4'])

input_video = cv2.VideoCapture(video_path[:-4]+'_'+str(100)+'fps.mp4')

# Get the video properties (frame width, frame height, and frames per second)
frame_width = int(input_video.get(3))
frame_height = int(input_video.get(4))
fps = int(input_video.get(5))
num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the codec and create a VideoWriter object to save the output video

output_video = cv2.VideoWriter(video_path[:-4]+'_'+str(100)+'fps_bg_sub.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height), isColor=True)

for i in tqdm(range(num_frames), 'Subtracting BG',miniters=40):
    ret, frame = input_video.read()

    # applying on each frame 
    fgmask = fgbg.apply(frame) 

    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
     # Invert the mask to keep the foreground
    fg = cv2.bitwise_and(frame, frame, mask=fgmask)

    # Save the frame to the output video
    output_video.write(fg)

# Release the video objects
input_video.release()
output_video.release()


Subtracting BG:   0%|          | 0/6196 [00:00<?, ?it/s]

In [40]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib

# Load the image
image_path = "screen04.png"
image = cv2.imread(image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Define a region of interest (ROI) covering the top half of the image
height, width, _ = image.shape
roi = image[height//4:height//4+int(height/2.5),int(width/7.5):int(width/7.5)+int(width/2.1)]

# Apply histogram equalization to the ROI
roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
#roi = cv2.addWeighted(roi, 2.5, np.zeros_like(roi), 0, 0)
#plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
#plt.title("Detected Side Faces")
#plt.axis('off')  # Turn off axis labels
#plt.show()
# Create a profile face cascade
half_body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')

# Adjust the minNeighbors and minSize parameters
profile_faces = half_body_cascade.detectMultiScale(roi, scaleFactor=1.01, minNeighbors=5, minSize=(100, 100), maxSize=(200,200))

# Draw rectangles around detected profile faces
for (x, y, w, h) in profile_faces:
    enlarger = 20
    x += int(width/7.5) ; y += height//4 + 20
    w = 80; h = 80
    image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)


# Display the image with detected side faces using Matplotlib
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Detected Side Faces")
plt.axis('off')  # Turn off axis labels
plt.show()


Using matplotlib backend: TkAgg


In [17]:
import xml.etree.ElementTree as ET

# Replace 'your_file.xml' with the actual XML file you want to read
xml_file = ANNOTATIONS_PATH+'/labels.xml'

# Parse the XML file
root = ET.parse(xml_file).getroot()

bones = []
max_indx = 0
# Explore the "Bone" elements
for bone in root.findall(".//Bone"):
    #print("Bone:")
    for index in bone.findall(".//Index"):
        low = index.get("Low")
        high = index.get("High")
        #print(f"  Index: Low={low}, High={high}")
        bones.append((int(low)-1,int(high)-1))
        if int(low) > max_indx or int(high) > max_indx:
            max_indx = max(int(low),int(high))

markers = []
# Explore the "Trajectory" elements
for trajectory in root.findall(".//Trajectory"):
    name = trajectory.find(".//Name").text
    #print(f"Trajectory: Name={name}")
    markers.append(name)

In [18]:
fullPosTableY = fullMarkersTable.iloc[:,3::3]
fullPosTableZ = fullMarkersTable.iloc[:,4::3]
fullPosTableX = fullMarkersTable.iloc[:,5::3]

fullPosTableX = fullPosTableX.rename(columns=dict(zip(list(fullPosTableX),[name.replace('Z','X').replace('Y','X') for name in list(fullPosTableX)])))
fullPosTableY = fullPosTableY.rename(columns=dict(zip(list(fullPosTableY),[name.replace('Z','Y').replace('X','Y') for name in list(fullPosTableY)])))
fullPosTableZ = fullPosTableZ.rename(columns=dict(zip(list(fullPosTableZ),[name.replace('X','Z').replace('Y','Z') for name in list(fullPosTableZ)])))

In [19]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


edges = np.array(bones)
adjacencyMatrix = np.zeros((max_indx,max_indx),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig = plt.figure(figsize=(960/100, 720/100), dpi=100)

# Add a 3D subplot with the specified position and size
ax = fig.add_axes([-0.05, -0.4, 1, 1.7], projection='3d')
ax.view_init(elev=10, azim=180)


minMax = np.zeros((2,3))
minMax[0,:] = [fullPosTableX.values.min(),fullPosTableY.values.min(),fullPosTableZ.values.min()]
minMax[1,:] = [fullPosTableX.values.max(),fullPosTableY.values.max(),fullPosTableZ.values.max()]

# Set appropriate axis limits
ax.set_xlim([minMax[0,0],minMax[1,0]])
ax.set_ylim([minMax[0,1],minMax[1,1]])
ax.set_zlim([minMax[0,2],minMax[1,2]])

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 1000
colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan']
plt.rcParams['grid.color'] = 'white'
ax.tick_params(axis='both', colors='w')
# Remove box faces except the ground
ax.w_xaxis.pane.fill = False
ax.w_yaxis.pane.fill = False
ax.w_zaxis.pane.fill = True
ax.w_zaxis.pane.set_facecolor((0.92,.92,.92))
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor((0,0,0))
# Hide axis spines and ticks
ax.w_xaxis.line.set_visible(False)
ax.w_yaxis.line.set_visible(True)
ax.w_zaxis.line.set_visible(False) 

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = fullPosTableX.iloc[val]
    filteredY = fullPosTableY.iloc[val]
    filteredZ = fullPosTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=10)

    # Add edges based on the weight matrix
    for i in range(max_indx):
        for j in range(i + 1, max_indx):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='k', linestyle='-', linewidth=0.7)

    ax.set_xlim([0,1])#minMax[0,0],minMax[1,0]])
    ax.set_ylim([0,1])#minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    
    #ax.set_xlabel('X', fontsize=12)
    #ax.set_ylabel('Y', fontsize=12)

    #ax.set_zlabel('Z', fontsize=12)
    #ax.set_title("Movement "+str(picked))
    #ax.grid(axis=['x','y'])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    #ax.set_xticklabels([])
    #ax.set_yticklabels([])
    #ax.set_zticklabels([])
    fig.canvas.draw_idle()
    
# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = fullPosTableX.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep <= maxValue:
            slider.set_val(slider.val + slider.valstep)
    elif event.button == 'up':
        if slider.val - slider.valstep >= 0:
            slider.set_val(slider.val - slider.valstep)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


/tmp/ipykernel_95952/3347424033.py:38: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax.w_xaxis.pane.fill = False
/tmp/ipykernel_95952/3347424033.py:39: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax.w_yaxis.pane.fill = False
/tmp/ipykernel_95952/3347424033.py:40: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax.w_zaxis.pane.fill = True
/tmp/ipykernel_95952/3347424033.py:41: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax.w_zaxis.pane.set_facecolor((0.92,.92,.92))
/tmp/ipykernel_95952/3347424033.py:46: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax.

In [20]:
import cv2
import os

# Open the video file
video_path = '/'.join(os.getcwd().split('/')[:-2]+['docs','presentation','resources','_'.join([folder,trial])+'.mp4'])
print(video_path)
cap = cv2.VideoCapture(video_path)

# Get the frames per second (FPS)
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(fps, cap.get(cv2.CAP_PROP_FRAME_COUNT))
ret, prev_frame = cap.read()
ret= not os.path.isfile(video_path[:-4]+'_'+str(fps * 2)+'fps.mp4')
num_frames = 0
if ret:
    output = cv2.VideoWriter(video_path[:-4]+'_'+str(fps * 2)+'fps.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps * 2, (int(cap.get(3)), int(cap.get(4))))
    while True:
        if num_frames < len(fullMarkersTable):
            output.write(prev_frame)
            num_frames += 1
        ret, frame = cap.read()
        if not ret:
            break
        averaged_frame = cv2.addWeighted(prev_frame, 0.5, frame, 0.5, 0)
        if num_frames < len(fullMarkersTable):
            output.write(averaged_frame)
            num_frames += 1
        else: break
        prev_frame = frame
    output.release()
else: 
    vidcap = cv2.VideoCapture(video_path[:-4]+'_'+str(fps * 2)+'fps_540p.mp4')
    print(vidcap.get(cv2.CAP_PROP_FRAME_COUNT), vidcap.get(3),vidcap.get(4))
print(num_frames)

# Release the video file
cap.release()


/home/gagg/Desktop/Thesis/OoM-Thesis/docs/presentation/resources/2016-05-26_t_006.mp4
50 3201.0
6196.0 960.0 540.0
0


In [53]:
from matplotlib import pyplot as plt
%matplotlib

edges = np.array(bones)
fullAdjacencyMatrix = np.zeros((max_indx,max_indx),dtype=bool)
fullAdjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig_full = plt.figure(figsize=(960/100, 540/100), dpi=100)

# Add a 3D subplot with the specified position and size
ax_full = fig_full.add_axes([-0.3, -0.2, 1.6, 1.6], projection='3d')
ax_full.view_init(elev=10, azim=180)


fullMinMax = np.zeros((2,3))
fullMinMax[0,:] = [fullPosTableX.values.min(),fullPosTableY.values.min(),fullPosTableZ.values.min()]
fullMinMax[1,:] = [fullPosTableX.values.max(),fullPosTableY.values.max(),fullPosTableZ.values.max()]

# Set appropriate axis limits
ax_full.set_xlim([fullMinMax[0,0],fullMinMax[1,0]])
ax_full.set_ylim([fullMinMax[0,1],fullMinMax[1,1]])
ax_full.set_zlim([fullMinMax[0,2],fullMinMax[1,2]])


plt.rcParams['grid.color'] = 'white'
ax_full.tick_params(axis='both', colors='w')
# Remove box faces except the ground
ax_full.w_xaxis.pane.fill = False
ax_full.w_yaxis.pane.fill = False
ax_full.w_zaxis.pane.fill = True
ax_full.w_zaxis.pane.set_facecolor((0.92,.92,.92))
ax_full.xaxis.pane.set_edgecolor('w')
ax_full.yaxis.pane.set_edgecolor('w')
ax_full.zaxis.pane.set_edgecolor((0,0,0))
# Hide axis spines and ticks
ax_full.w_xaxis.line.set_visible(False)
ax_full.w_yaxis.line.set_visible(True)
ax_full.w_zaxis.line.set_visible(False) 

Using matplotlib backend: TkAgg


/tmp/ipykernel_95952/1188419287.py:29: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax_full.w_xaxis.pane.fill = False
/tmp/ipykernel_95952/1188419287.py:30: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax_full.w_yaxis.pane.fill = False
/tmp/ipykernel_95952/1188419287.py:31: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_full.w_zaxis.pane.fill = True
/tmp/ipykernel_95952/1188419287.py:32: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_full.w_zaxis.pane.set_facecolor((0.92,.92,.92))
/tmp/ipykernel_95952/1188419287.py:37: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use 

In [54]:
reducedPosTableY = reducedMarkersTable.iloc[:,::3]
reducedPosTableZ = reducedMarkersTable.iloc[:,1::3]
reducedPosTableX = reducedMarkersTable.iloc[:,2::3]

reducedPosTableX = reducedPosTableX.rename(columns=dict(zip(list(reducedPosTableX),[name.replace('Z','X').replace('Y','X') for name in list(reducedPosTableX)])))
reducedPosTableY = reducedPosTableY.rename(columns=dict(zip(list(reducedPosTableY),[name.replace('Z','Y').replace('X','Y') for name in list(reducedPosTableY)])))
reducedPosTableZ = reducedPosTableZ.rename(columns=dict(zip(list(reducedPosTableZ),[name.replace('X','Z').replace('Y','Z') for name in list(reducedPosTableZ)])))

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

In [55]:
fig_reduced = plt.figure(figsize=(960/100, 540/100), dpi=100)

# Add a 3D subplot with the specified position and size
ax_reduced = fig_reduced.add_axes([-0.3, -0.2, 1.6, 1.6], projection='3d')
ax_reduced.view_init(elev=10, azim=180)


reducedMinMax = np.zeros((2,3))
reducedMinMax[0,:] = [reducedPosTableX.values.min(),reducedPosTableY.values.min(),reducedPosTableZ.values.min()]
reducedMinMax[1,:] = [reducedPosTableX.values.max(),reducedPosTableY.values.max(),reducedPosTableZ.values.max()]

# Set appropriate axis limits
ax_reduced.set_xlim([reducedMinMax[0,0],reducedMinMax[1,0]])
ax_reduced.set_ylim([reducedMinMax[0,1],reducedMinMax[1,1]])
ax_reduced.set_zlim([reducedMinMax[0,2],reducedMinMax[1,2]])


plt.rcParams['grid.color'] = 'white'
ax_reduced.tick_params(axis='both', colors='w')
# Remove box faces except the ground
ax_reduced.w_xaxis.pane.fill = False
ax_reduced.w_yaxis.pane.fill = False
ax_reduced.w_zaxis.pane.fill = True
ax_reduced.w_zaxis.pane.set_facecolor((0.92,.92,.92))
ax_reduced.xaxis.pane.set_edgecolor('w')
ax_reduced.yaxis.pane.set_edgecolor('w')
ax_reduced.zaxis.pane.set_edgecolor((0,0,0))
# Hide axis spines and ticks
ax_reduced.w_xaxis.line.set_visible(False)
ax_reduced.w_yaxis.line.set_visible(True)
ax_reduced.w_zaxis.line.set_visible(False) 

/tmp/ipykernel_95952/958718550.py:21: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax_reduced.w_xaxis.pane.fill = False
/tmp/ipykernel_95952/958718550.py:22: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax_reduced.w_yaxis.pane.fill = False
/tmp/ipykernel_95952/958718550.py:23: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_reduced.w_zaxis.pane.fill = True
/tmp/ipykernel_95952/958718550.py:24: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_reduced.w_zaxis.pane.set_facecolor((0.92,.92,.92))
/tmp/ipykernel_95952/958718550.py:29: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.

In [51]:

# Function to update the plot based on the slider value
def get_frame_full_skeleton(fig, ax, val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = fullPosTableX.iloc[val]
    filteredY = fullPosTableY.iloc[val]
    filteredZ = fullPosTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=10)

    # Add edges based on the weight matrix
    for i in range(len(fullAdjacencyMatrix)):
        for j in range(i + 1, len(fullAdjacencyMatrix)):
            if fullAdjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='k', linestyle='-', linewidth=1)

    ax.set_xlim([0,1])
    ax.set_ylim([0,1])
    ax.set_zlim([fullMinMax[0,2],fullMinMax[1,2]])

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    ncols, nrows = fig.canvas.get_width_height()
    frame = np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 3)

    return frame

def get_frame_reduced_skeleton(fig, ax, val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = reducedPosTableX.iloc[val]
    filteredY = reducedPosTableY.iloc[val]
    filteredZ = reducedPosTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=50)

    # Add edges based on the weight matrix
    for i in range(len(adjacencyMatrix)):
        for j in range(i + 1, len(adjacencyMatrix)):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='k', linestyle='-', linewidth=1.5)

    ax.set_xlim([0,1])
    ax.set_ylim([0,1])
    ax.set_zlim([reducedMinMax[0,2],reducedMinMax[1,2]])

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    ncols, nrows = fig.canvas.get_width_height()
    frame = np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 3)

    return frame

In [56]:
import cv2
from tqdm.notebook import tqdm

# Create a VideoWriter object in OpenCV
full_hd = (1920,1080)
out = cv2.VideoWriter(video_path[:-4]+'_'+str(fps * 2)+'fps_plus_skeleton.mp4', 
                      cv2.VideoWriter_fourcc(*'mp4v'), 
                      fps*2, 
                      full_hd)  # Adjust the filename, codec, frame rate, and frame size

vidcap = cv2.VideoCapture(video_path[:-4]+'_'+str(fps * 2)+'fps_540p.mp4')
filler_size = tuple(map(int,(vidcap.get(4),(full_hd[0]-vidcap.get(3))/2,3)))
filler = np.zeros(filler_size,dtype=np.uint8)+255
# Loop to update the plot and write frames to the video
for val in tqdm(range(len(fullMarkersTable)),'Saving video',miniters=20):  # Replace total_frames with the actual number of frames
    ret, video_frame = vidcap.read()
    if not ret:
        raise Exception('non matcha')
    
    # Draw an empty blue rectangle
    video_frame = cv2.rectangle(video_frame, (10, 10), (int(vidcap.get(3)-11), int(vidcap.get(4)-11)), (0, 0, 255), 2)
    video_frame = cv2.rectangle(video_frame, (0, 0), (int(vidcap.get(3)-2), int(vidcap.get(4)-2)), (127, 127, 127), 3)
    frame_full_skeleton = cv2.cvtColor(get_frame_full_skeleton(fig_full, ax_full, val), cv2.COLOR_RGB2BGR)
    frame_reduced_skeleton = cv2.cvtColor(get_frame_reduced_skeleton(fig_reduced, ax_reduced, val), cv2.COLOR_RGB2BGR)
    out.write(np.vstack((np.hstack((filler,video_frame,filler)), np.hstack((frame_full_skeleton,frame_reduced_skeleton)))))


# Release the VideoWriter and close the OpenCV window (if used)
out.release()
vidcap.release()

Saving video:   0%|          | 0/2596 [00:00<?, ?it/s]

In [86]:
reducedPosTableY = reducedMarkersTable.iloc[:,::3]
reducedPosTableZ = reducedMarkersTable.iloc[:,1::3]
reducedPosTableX = reducedMarkersTable.iloc[:,2::3]

reducedPosTableX = reducedPosTableX.rename(columns=dict(zip(list(reducedPosTableX),[name.replace('Z','X').replace('Y','X') for name in list(reducedPosTableX)])))
reducedPosTableY = reducedPosTableY.rename(columns=dict(zip(list(reducedPosTableY),[name.replace('Z','Y').replace('X','Y') for name in list(reducedPosTableY)])))
reducedPosTableZ = reducedPosTableZ.rename(columns=dict(zip(list(reducedPosTableZ),[name.replace('X','Z').replace('Y','Z') for name in list(reducedPosTableZ)])))

In [87]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True


plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')


minMax = np.zeros((2,3))
minMax[0,:] = [posTableX.values.min(),posTableY.values.min(),posTableZ.values.min()]
minMax[1,:] = [posTableX.values.max(),posTableY.values.max(),posTableZ.values.max()]

print(minMax)

# Set appropriate axis limits
ax.set_xlim([minMax[0,0],minMax[1,0]])
ax.set_ylim([minMax[0,1],minMax[1,1]])
ax.set_zlim([minMax[0,2],minMax[1,2]])

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0
colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan']
plt.rcParams['grid.color'] = 'white'
ax.tick_params(axis='both', colors='w')
# Remove box faces except the ground
ax.w_xaxis.pane.fill = False
ax.w_yaxis.pane.fill = False
ax.w_zaxis.pane.fill = True
ax.w_zaxis.pane.set_facecolor((0.92,.92,.92))
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor((0,0,0))
# Hide axis spines and ticks
ax.w_xaxis.line.set_visible(False)
ax.w_yaxis.line.set_visible(True)
ax.w_zaxis.line.set_visible(False) 

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=70)

    # Add edges based on the weight matrix
    for i in range(20):
        for j in range(i + 1, 20):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='gray', linestyle='-', linewidth=1.5)

    ax.set_xlim([0,1])#minMax[0,0],minMax[1,0]])
    ax.set_ylim([0,1])#minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    
    #ax.set_xlabel('X', fontsize=12)
    #ax.set_ylabel('Y', fontsize=12)
    #ax.set_zlabel('Z', fontsize=12)
    #ax.set_title("Movement "+str(picked))
    #ax.grid(axis=['x','y'])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    #ax.set_xticklabels([])
    #ax.set_yticklabels([])
    #ax.set_zticklabels([])
    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTableX.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep <= maxValue:
            slider.set_val(slider.val + slider.valstep)
    elif event.button == 'up':
        if slider.val - slider.valstep >= 0:
            slider.set_val(slider.val - slider.valstep)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg
[[0.         0.21972412 0.42422594]
 [0.72391286 1.         0.87679876]]


/tmp/ipykernel_2780/1515059041.py:42: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax.w_xaxis.pane.fill = False
/tmp/ipykernel_2780/1515059041.py:43: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax.w_yaxis.pane.fill = False
/tmp/ipykernel_2780/1515059041.py:44: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax.w_zaxis.pane.fill = True
/tmp/ipykernel_2780/1515059041.py:45: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax.w_zaxis.pane.set_facecolor((0.92,.92,.92))
/tmp/ipykernel_2780/1515059041.py:50: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax.w_xax

In [5]:
import numpy as np
jointNames = [col[:-2] for col in posTableX.columns]
adjacencyMatrix = np.zeros((len(jointNames), len(jointNames)), dtype=int)
links = [
    (0,1),(1,2),(0,2), # testa
    (7,36),(36,38),(38,40),(40,36),(7,38),(7,40),  # spalla dx
    (6,35),(6,37),(6,39),(37,39),(39,35),(35,37),   # spalla sx
    (40,34),(38,32),(34,30),(32,30),(34,32),    # braccio dx
    (37,31),(33,31),(33,39),(31,29),(33,29),    # braccio sx
    (4,11),(5,4),(12,5),(12,10),(11,10),    # bacino
    (10,9),(9,8),(12,7),(11,6),(3,4),(3,5),(3,6),(3,7),(1,8),(2,8),(8,6),(8,7), # busto
    (5,14),(4,13),(13,15),(13,16),(13,19),(14,18),(18,20),(14,17),(17,20),(16,19),(15,19),  # quadricipiti
    (17,24),(18,22),(16,21),(15,23),(24,22),(28,26),(26,22),    # caviglie parte 1
    (21,23),(23,27),(25,27),(25,21),(24,28),(20,24),(20,22),(19,23),(19,21),    # caviglie parte 2
    (10,16),(11,15),(12,17),(10,18) # piedi
]

for (fr,to) in links:
    adjacencyMatrix[fr,to] = 1
    adjacencyMatrix[to,fr] = 1
    adjacencyMatrix[fr+41,to+41] = 1
    adjacencyMatrix[to+41,fr+41] = 1

In [6]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

plt.close("all")
fig = plt.figure(figsize=(12,12))
sub_plot_position = [0, -0.2, 1, 1.5]
ax:plt.Axes = fig.add_subplot(sub_plot_position, projection='3d')
ax.set_box_aspect((2, 1, 0.6))
ax.view_init(elev=14, azim=-90)
ax.set_yticks(range(0,2000,500))
ax.set_zticks(range(0,1750,500))

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values)*0-2400,np.nanmin(posTableY.values)*0,np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values)*0+2700,np.nanmax(posTableY.values)*0+2000,np.nanmax(posTableZ.values)]

#minMax[0,:] = [np.nanmin(posTableX.iloc[:1,:].values),np.nanmin(posTableY.iloc[:1,:].values),np.nanmin(posTableZ.iloc[:1,:].values)]
#minMax[1,:] = [np.nanmax(posTableX.iloc[:1,:].values),np.nanmax(posTableY.iloc[:1,:].values),np.nanmax(posTableZ.iloc[:1,:].values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=5,c="r")

    #for i, col_name in enumerate(posTableX.columns):
    #    ax.text(filteredX[i], filteredY[i], filteredZ[i], str(i), horizontalalignment='center',fontsize=6, color='black')

    # Draw lines based on the adjacency matrix
    for i in range(adjacencyMatrix.shape[0]):
        for j in range(i + 1, adjacencyMatrix.shape[1]):
            if adjacencyMatrix[i, j] == 1:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]], color='black',linewidth=0.9)

    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    ax.set_yticks(range(0,2000,500))
    ax.set_zticks(range(0,1750,500))
    
    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


# Cutting Videos

## New Dataset

In [ ]:
try:
    import cv2
    import tdqm
except:
    %pip install opencv-python
    %pip install tdqm
    import cv2

In [2]:
from os.path import join, exists
from os import makedirs
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 8, 8,10,11,13,15,17,12,14,16,18,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 9,10,18,13,15,17,18,14,16,19,19,20]
edges = np.array(list(zip(jointsFrom,jointsTo)))
edgeNames = [
    'Rfoot-Rankle',         # 1-3
    'Rankle-Rknee',         # 3-5      
    'Rknee-Rhip',           # 5-7
    'Rhip-Chip',            # 7-8
    'Lfoot-Lankle',         # 2-4
    'Lankle-Lknee',         # 4-6      
    'Lknee-Lhip',           # 6-9
    'Lhip-Chip',            # 8-9         
    'Chip-spine',           # 8-10
    'spine-Cshoulder',      #10-18
    'Rhand-Rwrist',         #11-13
    'Rwrist-Relbow',        #13-15
    'Relbow-Rshoulder',     #15-17  
    'Rshoulder-Cshoulder',  #17-18   
    'Lhand-Lwrist',         #12-14
    'Lwrist-Lelbow',        #14-16
    'Lelbow-Lshoulder',     #16-19
    'Lshoulder-Cshoulder',  #18-19  
    'Cshoulder-head']       #18-20

PATH = '/media/gagg/SSD/OoM/MoCap/Wholodance/compressed_videos'     # local path of mocap videos, change accordingly to structure
OUTPUT_ROOT_FOLDER = PATH+'/cut_videos'     # creates a child folder
VIDEOS_SUFFIX = '_video01.mp4'     # frontal view
EXTRA_SECONDS_PRE_MOVEMENT = 1     # adds extra seconds to resulting video for better observe OoMs
EXTRA_SECONDS_POST_MOVEMENT = 2    # adds extra seconds to resulting video for better observe OoMs
OVERWRITE = True                   # set to true to overwrite cut videos if already present

# annotations are formatted as a first line of header but it's unused but must be followed as if it was a csv
# folderSrc, trialId, fragId, jointIdOOM, startSecond, endSecond, [optionals: (ok-check, description)]
with open('/'.join(PATH.split('/')[:-2])+'/annotations.txt','r') as file:
    annotations = file.read().replace(' ','').split('\n')[1:-1]

#picking just the validated manually
annotations = [ann for ann in annotations if 'ok' in ann]
must_rewind_video = True
for i,ann in tqdm(enumerate(annotations), total=len(annotations), desc="Processing videos",ncols=int(np.clip(len(annotations)*5,0,150))):
    ann = ann.split(',')[:6]

    folder,name_file,fragId,OoM_edges,start_time,end_time = ann[0],ann[1],int(ann[2]),[tuple(map(int, ann[3].strip('[]').split(';')[i].split('-'))) for i in range(0, int(';' in ann[3])+1) ],float(ann[4]),float(ann[5])
    must_open_new_video = i == 0 or fragId == 0 or start_time-EXTRA_SECONDS_PRE_MOVEMENT <= float(annotations[i-1].split(',')[5])+EXTRA_SECONDS_POST_MOVEMENT
    must_close_video = i == len(annotations)-1 or int(annotations[i+1].split(',')[2])==0 or end_time+EXTRA_SECONDS_POST_MOVEMENT >= float(annotations[i+1].split(',')[4])-EXTRA_SECONDS_PRE_MOVEMENT

    if must_open_new_video:
        frame_number = 0
        video_path = join(PATH,folder,folder+'_'+name_file+VIDEOS_SUFFIX)
        cap = cv2.VideoCapture(video_path)

    # Check if the video file was opened successfully
    if not cap.isOpened():
        print("Error opening video file")
        exit()

    # Get the frames per second (fps) of the video
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Calculate the frame indexes for start and end times
    start_frame = int(np.clip(start_time-EXTRA_SECONDS_PRE_MOVEMENT,0.,np.inf) * fps)
    end_frame = int(np.clip(end_time+EXTRA_SECONDS_POST_MOVEMENT,0.,np.floor(cap.get(cv2.CAP_PROP_FRAME_COUNT)/fps)-1) * fps)

    # Read and save the frames within the specified range
    frames = []
    while True:
        _, frame = cap.read()
        if start_frame <= frame_number <= end_frame:
            cv2.putText(img=frame,
                        text=' / '.join([edgeNames[i] for i in np.where(np.all(edges[:,np.newaxis,:] == OoM_edges, axis=2))[0].tolist()]), 
                        org=(int(10+3*np.floor(np.max(frame.shape)/480)),int(20+8*np.floor(np.max(frame.shape)/480))), 
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                        fontScale=0.5*np.floor(np.max(frame.shape)/480), 
                        color=(255,255,255), 
                        thickness=1)
            if start_time*fps <= frame_number <= end_time*fps:
                cv2.rectangle(img=frame, pt1=(5,5), pt2=(frame.shape[1]-5,frame.shape[0]-5), color=(0, 0, 255), thickness=np.clip(int(np.floor(np.max(frame.shape)/480)),1,np.inf).astype(int))
            frames.append(frame)
        elif frame_number > end_frame:
            break
        frame_number += 1
    if must_close_video:
        cap.release()

    # Save the frames as a new video
    output_folder = join(OUTPUT_ROOT_FOLDER,folder)

    if not exists(output_folder): makedirs(output_folder)
    
    output_path = join(output_folder,folder+'_'+name_file+'_frag'+str(fragId)+'.mp4')
    if not exists(output_path) or OVERWRITE:
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frames[0].shape[1], frames[0].shape[0]))
        for frame in frames:
            out.write(frame)
        out.release()

Processing videos: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [03:03<00:00,  4.36s/it]


## Old Dataset

Write annotations

In [12]:
from itertools import groupby, chain
import numpy as np
import re

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 8, 8,10,11,13,15,17,12,14,16,18,18,21]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 9,10,18,13,15,17,18,14,16,19,19,20,21]
edges = np.array(list(zip(jointsFrom,jointsTo)))
edgeNames = [
    'Rfoot-Rankle',         # 1-3
    'Rankle-Rknee',         # 3-5      
    'Rknee-Rhip',           # 5-7
    'Rhip-Chip',            # 7-8
    'Lfoot-Lankle',         # 2-4
    'Lankle-Lknee',         # 4-6      
    'Lknee-Lhip',           # 6-9
    'Lhip-Chip',            # 8-9         
    'Chip-spine',           # 8-10
    'spine-Cshoulder',      #10-18
    'Rhand-Rwrist',         #11-13
    'Rwrist-Relbow',        #13-15
    'Relbow-Rshoulder',     #15-17  
    'Rshoulder-Cshoulder',  #17-18   
    'Lhand-Lwrist',         #12-14
    'Lwrist-Lelbow',        #14-16
    'Lelbow-Lshoulder',     #16-19
    'Lshoulder-Cshoulder',  #18-19  
    'Cshoulder-head',       #18-20
    'No-OoM']       
y_mask =     {1:    [edgeNames.index('Rshoulder-Cshoulder')],
              2:    [edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder'),edgeNames.index('Cshoulder-head')],
              3:    [edgeNames.index('Rhip-Chip')],
              4:    [edgeNames.index('Lshoulder-Cshoulder')],
              5:    [edgeNames.index('Cshoulder-head')],
              6:    [edgeNames.index('Rankle-Rknee')],
              7:    [edgeNames.index('Lshoulder-Cshoulder')],
              8:    [edgeNames.index('Rshoulder-Cshoulder')],
              9:    [edgeNames.index('Rankle-Rknee'),edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip')],
              10:   [edgeNames.index('Lshoulder-Cshoulder')],
              11:   [edgeNames.index('Rhand-Rwrist')],
              12:   [edgeNames.index('Relbow-Rshoulder'),edgeNames.index('Rshoulder-Cshoulder'),edgeNames.index('Cshoulder-head')],
              13:   [edgeNames.index('Lankle-Lknee'),edgeNames.index('Lknee-Lhip')],
              14:   [edgeNames.index('Lknee-Lhip')],
              15:   [edgeNames.index('Lwrist-Lelbow'),edgeNames.index('Lelbow-Lshoulder')],
              16:   [edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              17:   [edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Lknee-Lhip')],
              18:   [edgeNames.index('No-OoM')],
              19:   [edgeNames.index('No-OoM')],
              20:   [edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              21:   [edgeNames.index('Relbow-Rshoulder')],
              22:   [edgeNames.index('Lelbow-Lshoulder'),edgeNames.index('Lshoulder-Cshoulder')],
              23:   [edgeNames.index('Lelbow-Lshoulder'),edgeNames.index('Lshoulder-Cshoulder')],
              24:   [edgeNames.index('Rwrist-Relbow')],
              25:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Rknee-Rhip')],
              26:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              27:   [edgeNames.index('Lshoulder-Cshoulder'),edgeNames.index('Lelbow-Lshoulder')],
              28:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Rknee-Rhip')],
              29:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Rknee-Rhip')],
              30:   [edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              31:   [edgeNames.index('Rhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              32:   [edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder'),edgeNames.index('Rshoulder-Cshoulder'),edgeNames.index('Relbow-Rshoulder')],
              33:   [edgeNames.index('Lelbow-Lshoulder'),edgeNames.index('Lshoulder-Cshoulder'),edgeNames.index('Rshoulder-Cshoulder'),edgeNames.index('Relbow-Rshoulder')],
              34:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Rknee-Rhip')],
              35:   [edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder'),edgeNames.index('Rshoulder-Cshoulder')],
              36:   [edgeNames.index('Relbow-Rshoulder')]
              }

secondsMap = { 1:  (37,40.5),        
      2:  (58,64),        
      3:  (73,76),
      4:  (13,16),
      5:  (36.3,41.5),
      6:  (44.4,50),
      7:  (3,8.9),
      8:  (4,9),
      9:  (9.2,13.4),
      10: (14,20.1),
      11: (4.7,8.3),
      12: (12.8,18.5),
      13: (24.2,29.7),
      14: (29.8,36.2),
      15: (51.1,55),
      16: (57.9,63.8),
      17: (16.9,18.7),
      18: (11.5,13.2),
      19: (15,18),
      20: (19,21.5),
      21: (2.6,5),
      22: (58,61),
      23: (25.5,28.5),
      24: (2.5,5.5),
      25: (8.1,11.8),       # ball catch left side
      26: (11.8,15.1),      # ball catch right side
      27: (12.15,14.7),     # ball throw left side
      28: (8.5,11.8),       # ball throw right side
      29: (22,24.5),        # ball high throw left side
      30: (22,24.5),        # ball side catch right side
      31: (25,27.2),        # ball throw left side
      32: (25,27.2),        # ball catch right side
      33: (29,32.1),       # ball throw cheat left side
      34: (29,32.1),       # ball catch right side
      35: (74,80),         # right ball catch left side 
      36: (74,80)          # ball catch right side
    }
PATH = '/media/gagg/SSD/OoM/OlgaMatth_Thesis/Recordings_OM'
FOLDERS =   ['Wholodance/2016-03-21 (Muriel)']*4+ \
            ['Wholodance/2016-03-22 (Marianne)']*6+ \
            ['Wholodance/2019-05-15 (Cora)']*6+ \
            ['Montpellier-Unige-Dataset/2020-07-07/morning/videos']*8+ \
            ['Montpellier-Unige-Dataset/2020-07-08/morning/videos']*8+ \
            ['Montpellier-Unige-Dataset/2020-07-13/morning/videos']*4
TRIALS = ['t_003']*3+['t_004']+['t_002']*2+['t_003','t_005','t_005','t_012']+['t_028']*6+['t_051','t_052','t_052','t_052','t_060','t_061','t_063','t_064','t_052','t_052','t_052','t_052','t_053','t_053','t_053','t_053','t_061','t_061','t_061','t_061']
FRAGS = list(chain.from_iterable([list(range(len(list(group)))) for _, group in groupby(TRIALS)]))
pattern = r'\(.*\)'
with open(PATH+'/annotations.txt','w') as file:
    file.write('folderSrc, trialId, fragId, jointIdOoM, startSecond, endSecond, (check, side, description)\n')
    for i,elem in enumerate(y_mask.keys()):
        file.write(', '.join([FOLDERS[i].split('/')[1].replace(re.findall(pattern,FOLDERS[i].split('/')[1])[0],'').replace(' ','') if re.findall(pattern,FOLDERS[i].split('/')[1]) else FOLDERS[i].split('/')[1],
                             str(TRIALS[i].replace('_','')),
                             str(FRAGS[i]),
                             '['+'-'.join(map(str, edges[y_mask[elem][0]].tolist()))+(';'+'-'.join(map(str, edges[y_mask[elem][-1]].tolist())))*( edges[y_mask[elem][0]].tolist() != edges[y_mask[elem][-1]].tolist())+']',
                             str(float(secondsMap[elem][0])),
                             str(float(secondsMap[elem][1])),
                             'ok' if (TRIALS[i] != 't_052' or not '2020-07-08' in FOLDERS[i]) else "manca video intero"])+'\n')


Read annotations

In [22]:
import re
from os.path import join, exists
from os import makedirs
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 8, 8,10,11,13,15,17,12,14,16,18,18,21]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 9,10,18,13,15,17,18,14,16,19,19,20,21]
edges = np.array(list(zip(jointsFrom,jointsTo)))
edgeNames = [
    'Rfoot-Rankle',         # 1-3
    'Rankle-Rknee',         # 3-5      
    'Rknee-Rhip',           # 5-7
    'Rhip-Chip',            # 7-8
    'Lfoot-Lankle',         # 2-4
    'Lankle-Lknee',         # 4-6      
    'Lknee-Lhip',           # 6-9
    'Lhip-Chip',            # 8-9         
    'Chip-spine',           # 8-10
    'spine-Cshoulder',      #10-18
    'Rhand-Rwrist',         #11-13
    'Rwrist-Relbow',        #13-15
    'Relbow-Rshoulder',     #15-17  
    'Rshoulder-Cshoulder',  #17-18   
    'Lhand-Lwrist',         #12-14
    'Lwrist-Lelbow',        #14-16
    'Lelbow-Lshoulder',     #16-19
    'Lshoulder-Cshoulder',  #18-19  
    'Cshoulder-head',       #18-20
    'No-OoM'] 

PATH = '/media/gagg/SSD/OoM/OlgaMatth_Thesis/Recordings_OM/compressed_videos'
OUTPUT_ROOT_FOLDER = PATH+'/cut_videos'
VIDEOS_SUFFIX = '_video01.mp4' 
EXTRA_SECONDS_PRE_MOVEMENT = 1     # adds extra seconds to resulting video for better observe OoMs
EXTRA_SECONDS_POST_MOVEMENT = 2    # adds extra seconds to resulting video for better observe OoMs
OVERWRITE = True                   # set to true to overwrite cut videos if already present

with open('/'.join(PATH.split('/')[:-1])+'/annotations.txt','r') as file:
    annotations = file.read().splitlines()[1:]
annotations = [ann for ann in annotations if not "manca video intero" in ann]

must_rewind_video = True
pattern = r'\(.*\)'
for i,ann in tqdm(enumerate(annotations), total=len(annotations), desc="Processing videos",ncols=int(np.clip(len(annotations)*5,0,150))):
    ann = ann.replace(' ','').split(',')[:6]

    folder,trial,fragId,OoM_edges,start_time,end_time = ann[0],ann[1],int(ann[2]),[tuple(map(int, ann[3].strip('[]').split(';')[i].split('-'))) for i in range(0, int(';' in ann[3])+1) ],float(ann[4]),float(ann[5])
    must_open_new_video = i == 0 or fragId == 0 or start_time-EXTRA_SECONDS_PRE_MOVEMENT <= float(annotations[i-1].split(',')[5])+EXTRA_SECONDS_POST_MOVEMENT
    must_close_video = i == len(annotations)-1 or int(annotations[i+1].split(',')[2])==0 or end_time+EXTRA_SECONDS_POST_MOVEMENT >= float(annotations[i+1].split(',')[4])-EXTRA_SECONDS_PRE_MOVEMENT

    if must_open_new_video:
        frame_number = 0
        video_path = join(PATH,folder+'_'+trial+VIDEOS_SUFFIX)
        cap = cv2.VideoCapture(video_path)

    # Check if the video file was opened successfully
    if not cap.isOpened():
        raise Exception("Error opening video file")

    # Get the frames per second (fps) of the video
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Calculate the frame indexes for start and end times
    start_frame = int(np.clip(start_time-EXTRA_SECONDS_PRE_MOVEMENT,0.,np.inf) * fps)
    end_frame = int(np.clip(end_time+EXTRA_SECONDS_POST_MOVEMENT,0.,np.floor(cap.get(cv2.CAP_PROP_FRAME_COUNT)/fps)-1) * fps)

    # Read and save the frames within the specified range
    frames = []
    while True:
        _, frame = cap.read()
        if start_frame <= frame_number <= end_frame:
            cv2.putText(img=frame,
                        text=' / '.join([edgeNames[i] for i in np.where(np.all(edges[:,np.newaxis,:] == OoM_edges, axis=2))[0].tolist()]), 
                        org=(int(10+3*np.floor(np.max(frame.shape)/480)),int(20+8*np.floor(np.max(frame.shape)/480))), 
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                        fontScale=0.5*np.floor(np.max(frame.shape)/480), 
                        color=(255,255,255), 
                        thickness=1)
            if start_time*fps <= frame_number <= end_time*fps:
                cv2.rectangle(img=frame, pt1=(5,5), pt2=(frame.shape[1]-5,frame.shape[0]-5), color=(0, 0, 255), thickness=np.clip(int(np.floor(np.max(frame.shape)/480)),1,np.inf).astype(int))
            frames.append(frame)
        elif frame_number > end_frame:
            break
        frame_number += 1
    if must_close_video:
        cap.release()

    # Save the frames as a new video
    output_folder = join(OUTPUT_ROOT_FOLDER,folder)

    if not exists(output_folder): makedirs(output_folder)
    
    output_path = join(output_folder,folder+'_'+trial+'_frag'+str(fragId)+'.mp4')
    if not exists(output_path) or OVERWRITE:
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frames[0].shape[1], frames[0].shape[0]))
        for frame in frames:
            out.write(frame)
        out.release()      

Processing videos:   0%|                                                                                                       | 0/32 [00:00<?, ?it/s]

Processing videos: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [02:47<00:00,  5.23s/it]
